<center> <img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>    
<table>
    <th><font size=5>Специализация "Машинное обучение и анализ данных" от:</font></th>
    <th><img src="https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://d15cw65ipctsrr.cloudfront.net/11/ae0000b18911e5965623dd71776f15/mipt.png?auto=format%2Ccompress&dpr=1&w=200&h=100&fit=clip"/> </th>
    <th><img src="https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://d15cw65ipctsrr.cloudfront.net/fe/ab6f503b2b11e796ddbfacaf40b9e6/Yandex.jpg?auto=format%2Ccompress&dpr=1&w=200&h=100&fit=clip"/></th>
</table>

# <center> ПРОЕКТ: Идентификация пользователей по посещенным веб-страницам
# <center> АВТОР ПРОЕКТА: Дмитрий Шерешевский, *PhD*
##    
<center>Исходный материал: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

<img src='http://smartcity.eletsonline.com/wp-content/uploads/2017/04/digitalindia.png'>

# <center>Часть 6 из 6.  Vowpal Wabbit

В этой части мы применим популярную библиотеку **Vowpal Wabbit** и попробуем ее на данных соревнования. Перед этим поработаем с  [данными](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) Scikit-learn по новостям, сначала в режиме бинарной классификации, затем – в многоклассовом режиме. Затем будем классифицировать рецензии к фильмам с сайта IMDB. Наконец, применим Vowpal Wabbit к данным по веб-сессиям. Vowpal Wabbit - отличная библиотека!

**План 6 части:**
- 6.1. Статья по Vowpal Wabbit
- 6.2. Применение Vowpal Wabbit к данным по посещению сайтов
 - 2.1. Подготовка данных
 - 2.2. Валидация по отложенной выборке
 - 2.3. Валидация по тестовой выборке (Public Leaderboard)
- 6.3. Vowpal Wabbit в соревновании
 - Введение
 - 3.1. Функция для подготовки признаков из "сырых" данных
 - 3.2. Валидация по отложенной выборке
 - 3.3. Подбор гиперпараметров

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

## <center>6.1. Статья про Vowpal Wabbit
Краткая обзорная статья по Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению [здесь](https://habrahabr.ru/company/ods/blog/326418/). Можно скачать [ноутбук](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_russian/topic08_sgd_hashing_vowpal_wabbit/topic8_sgd_hashing_vowpal_wabbit.ipynb), прилагаемый к статье, и попробовать на практике разобраться с Vowpal Wabbit.

## <center>6.2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы, но уже в задаче классификации на 400 классов будет хорошо заметна скорость работы VW. Данные [здесь](https://inclass.kaggle.com/c/identify-me-if-you-can4/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [5]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

In [6]:
# Путь к данным
PATH_TO_DATA = ('../data')

**Загрузим обучающую и тестовую выборки. Можно заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [53]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [5]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [54]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


In [7]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder` и затем еще +1 добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [8]:
y = train_df_400['user_id']
class_encoder = LabelEncoder()
y_for_vw = class_encoder.fit_transform(y) + 1

**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объединим обучающиую и тестовую выборки
- выберем только сайты (признаки от 'site1' до 'site10')
- заменим пропуски на нули (сайты у нас нумеровались с 0)
- переведём в разреженный формат `csr_matrix`
- разобьем обратно на обучающую и тестовую части

In [9]:
sites = ['site' + str(i) for i in range(1, 11)]

In [10]:
train_test_df = pd.concat([train_df_400, test_df_400])
train_test_df_sites = train_test_df[sites].fillna(0).astype(int)
train_test_sparse = csr_matrix((np.ones(train_test_df_sites.values.size), 
                                train_test_df_sites.values.ravel(), 
                                np.arange(train_test_df_sites.values.shape[0] + 1) * 
                                train_test_df_sites.values.shape[1]), dtype=int)[:, 1:]

X_train_sparse = train_test_sparse[:train_df_400.shape[0]]
X_test_sparse = train_test_sparse[train_df_400.shape[0]:]
y = train_df_400['user_id'].values

### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [11]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].iloc[:train_share, :]
valid_df = train_df_400[sites].iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [12]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

**Реализуем функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [83]:
def arrays_to_vw(X, y=None, train=True, out_file=os.path.join(PATH_TO_DATA,'tmp.vw')):
    if y is None:
        y = [1] * X.shape[0]
    with open(out_file, 'w') as vw_data:
        for row, site_id in zip(X, y):
            sample = str(site_id) +' | ' + ' '.join(row.astype(int).astype(str)) + '\n'
            vw_data.write(sample)
    pass

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратим внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [15]:
%%time
# будет 4 вызова
arrays_to_vw(train_df_part.fillna(0).values, 
             y_train_part_for_vw, 
             out_file=os.path.join(PATH_TO_DATA,'train_part.vw'))

arrays_to_vw(valid_df.fillna(0).values, 
             y_valid_for_vw, 
             out_file=os.path.join(PATH_TO_DATA,'valid.vw'))

arrays_to_vw(train_df_400[sites].fillna(0).values, 
             y_for_vw, 
             out_file=os.path.join(PATH_TO_DATA,'train.vw'))

arrays_to_vw(test_df_400[sites].fillna(0).values, 
             out_file=os.path.join(PATH_TO_DATA,'test.vw'))

Wall time: 9.08 s


**Проверяем:**

In [16]:
!head -3 $PATH_TO_DATA/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


**Обучим модель Vowpal Wabbit на выборке `train_part.vw`. Укажем, что решается задача классификации с 400 классами (`--oaa`), сделаем 3 прохода по выборке (`--passes`). Зададим некоторый кэш-файл (флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажем значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажем `random_seed`=17. Остальные параметры пока оставим по умолчанию.**

In [19]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [20]:
%%time
!vw --oaa 400 -d $train_part_vw --passes 3 -c -k \
-f $model -b 26 --random_seed 17

Wall time: 1min 31s


final_regressor = data/vw_model.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = data/train_part.vw.cache
Reading datafile = data/train_part.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976563 0.984375          128          128.0      348      346       11
0.941406 0.906250          256          2

**Запишем прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [55]:
%%time
!vw -t -i $model \
-d $valid_vw \
-p $PATH_TO_DATA/vw_valid_pred.csv --random_seed 17

Wall time: 2.91 s


only testing
predictions = data/vw_valid_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4      188       11
1.000000 1.000000            2            2.0      160      220       11
0.750000 0.500000            4            4.0      143      143       11
0.750000 0.750000            8            8.0      247      247       11
0.687500 0.625000           16           16.0      341       30       11
0.593750 0.500000           32           32.0      237      237       11
0.609375 0.625000           64           64.0      178      178       11
0.640625 0.671875          128          128.0      132      228       11
0.656250 0.671875          256          256.0       14       14       11
0.646484 0.636

**Считаем прогнозы *"../data/vw_valid_pred.csv"*  из файла и посмотрим на долю правильных ответов на отложенной части.**

In [72]:
from sklearn.metrics import accuracy_score

vw_valid_pred = np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_valid_pred.csv'))

accuracy_score(y_valid_for_vw, vw_valid_pred)

0.34541741128414605

**Теперь обучим `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделаем прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитаем доли верных ответов. Логистическая регрессия обучается долго – это нормально. Укажем везде `random_state`=17, `n_jobs`=-1.**

In [79]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
sgd_logit = SGDClassifier(loss='log', max_iter=3, random_state=17, n_jobs=-1)

In [80]:
%%time
logit.fit(X_train_part_sparse, y_train_part)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


Wall time: 30min 13s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [80]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

Wall time: 12 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

**Какими получаются доли правильных ответов на отложенной выборке для Vowpal Wabbit, SGD и логистической регрессии? Запишите 3 числа в файл *answer6_1.txt* (в указанном порядке, сначала VW, потом SGD и logit) через пробел, округлив до 3 знаков после запятой.**

In [82]:
vw_valid_acc = accuracy_score(y_valid_for_vw, vw_valid_pred)
sgd_valid_acc = accuracy_score(y_valid, sgd_logit.predict(X_valid_sparse))
logit_valid_acc = accuracy_score(y_valid, logit.predict(X_valid_sparse))

In [85]:
print("vw_accuracy = {}, sgd_accuracy = {}, logit_accuracy = {}"
      .format(round(vw_valid_acc, 3), round(sgd_valid_acc, 3), round(logit_valid_acc, 3)))

vw_accuracy = 0.345, sgd_accuracy = 0.291, logit_accuracy = 0.363

### 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучим модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [28]:
%%time
!vw --oaa 400 -d $train_vw --passes 3 -c -k \
-f $PATH_TO_DATA/vw_model_400_train.vw -b 26 --random_seed 17

Wall time: 1min 35s


final_regressor = data//vw_model_400_train.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = data/train.vw.cache
Reading datafile = data/train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976563 0.984375          128          128.0      348      346       11
0.941406 0.906250          256          

**Сделаем прогноз для тестовой выборки.**

In [34]:
%%time
!vw -t -i $PATH_TO_DATA/vw_model_400_train.vw \
-d $test_vw \
-p $PATH_TO_DATA/vw_400_test_pred.csv --random_seed 17

Wall time: 1.76 s


only testing
predictions = data//vw_400_test_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        1       90       11
1.000000 1.000000            2            2.0        1       21       11
1.000000 1.000000            4            4.0        1      265       11
1.000000 1.000000            8            8.0        1      137       11
1.000000 1.000000           16           16.0        1      273       11
1.000000 1.000000           32           32.0        1      384       11
1.000000 1.000000           64           64.0        1      139       11
1.000000 1.000000          128          128.0        1       85       11
1.000000 1.000000          256          256.0        1       25       11
0.994141 0.

**Запишем прогноз в файл, применим обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправим решение на Kaggle.**

In [86]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [41]:
vw_pred = class_encoder.inverse_transform(np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_400_test_pred.csv'), dtype=int) - 1)

In [46]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users1.csv'))

**Сделаем то же самое для SGD и логистической регрессии. Здесь времени потребуется гораздо больше.**

In [81]:
logit = LogisticRegression(random_state=17)
sgd_logit = SGDClassifier(loss='log', max_iter=3, random_state=17, n_jobs=-1)

In [32]:
%%time
logit.fit(X_train_sparse, y)

Wall time: 1h 47min 55s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [82]:
%%time
sgd_logit.fit(X_train_sparse, y)

Wall time: 17.1 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=3, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [43]:
logit_test_pred = logit.predict(X_test_sparse)
sgd_logit_test_pred = sgd_logit.predict(X_test_sparse)

In [45]:
write_to_submission_file(logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users1.csv'))
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users1.csv'))

Получим следующие доли правильных ответов  **Vowpal Wabbit, SGD и логистической регрессии** на публичной части (public leaderboard) тестовой выборки [этого](https://inclass.kaggle.com/c/identify-me-if-you-can4): 

0.188   
0.175   
0.199

**ВЫВОД по соотношению качества классификации и скорости обучения VW, SGD и logit:**
- видим, что **Vowpal Wabbit** работает **значительно быстрее**, чем SGD и особенно логистическая регрессия, в то время как качество классификации - немного хуже логистической регрессии, но лучше SGD.

## Пути улучшения

Что еще можно попробовать в [соревновании](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2):
 - Использовать ранее построенные признаки для улучшения модели
 - Настроить параметры Vowpal Wabbit с hyperopt
 - Попробовать TF-IDF и n-граммы

## <center> Часть 3. Vowpal Wabbit в соревновании

### Введение

Применим полученный опыт работы с **Vowpal Wabbit** для получения результата в [соревновании](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2).   

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [4]:
# Путь к данным
PATH_TO_DATA = ('data')

In [6]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [7]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


Отметим, что для работы с Vowpal Wabbit благодаря **хешированию** объединять обучающую и тестовую выборки нам нет необходимости.

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно. Поэтому для соревнования выбрана метрика **roc_auc**.

In [8]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

### 3.1. Функция для подготовки признаков из "сырых" данных

Сделаем функцию, формирующую из загруженных выше "сырых" данных признаки, которые мы впоследствии преобразуем к формату для Vowpal Wabbit.    

Будем это делать на основе опыта, полученного на 5 недели, когда мы участвовали в соревновании с использованием линейного  классификатора **SGDClassifier** и классификатора на основе градиентного бустинга из библиотеки **XGBoost**. Учтем все **инсайты**, к которым мы пришли. Будем придерживаться следующего:
- сформируем функцию для признаков из исходноых данных на базе итоговой функции из 5 недели
- учтем, что все дополнительные признаки - по сути **категориальные**
- сформируем признаки, которые хорошо себя зарекомендовали на предыдущем этапе
- не будем пока добавлять новых признаков, чтобы иметь возможность сравнить алгоритмы.
- адаптируем функцию к последующему преобразованию в формат Vowpal Wabbit.

**Итак, сформируем функцию для признаков. На вход она должна принимать "сырой" датафрейм с данными, а на выходе должен быть датафрейм с признаками и целевой вектор `y`, адапрированный по Vopal Wabbit, то есть принимающий зачения [-1, 1]. Пропуски заменим на 0.**

In [9]:
def feature_engineering_df_vw(train_df, test=False, features=None):
    '''
    features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    '''
    if features is None:
        features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    
    df = train_df
    if test:
        df['target'] = 1
    else:
        df['target'] = df['target'].map({1:1, 0:-1})
    
    
    # подготовим временнЫе метки
    df[['time%d' % i for i in range(1, 11)]] = df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        df['#unique_sites'] = df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
        df['session_timespan'] = df['session_timespan'].apply(lambda x: int(x // 30) if x < 210 else 7)
    if 'start_hour' in features:    
        df['start_hour'] = df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        df['day_of_week'] = df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        df['time_of_day'] = df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        df['weekend'] = df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
        
    col = ['site%d' % i for i in range(1, 11)] + features
    
    return (df[col].fillna(0).astype(int), df['target'].values.astype(int)) if not test else df[col].fillna(0).astype(int)

Проверяем функцию:

In [10]:
%%time
train_for_vw_testing, y = feature_engineering_df_vw(train_df.head().copy())

Wall time: 34.1 ms


In [11]:
train_for_vw_testing

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,#unique_sites,session_timespan,start_hour,day_of_week,time_of_day,weekend
session_id,,,,,,,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0,1,0,10,3,1,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518,7,0,11,5,1,1
3,14769,39,14768,14769,37,39,14768,14768,14768,14768,4,0,16,0,2,0
4,782,782,782,782,782,782,782,782,782,782,1,7,10,4,1,0
5,22,177,175,178,177,178,175,177,177,178,4,7,10,4,1,0


Применим функцию к обучающей и тестовой выборкам.

In [13]:
%%time
train_for_vw, y_for_vw = feature_engineering_df_vw(train_df)

Wall time: 4min 47s


In [14]:
%%time
test_for_vw = feature_engineering_df_vw(test_df, test=True)

Wall time: 1min 34s


In [15]:
train_for_vw.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,#unique_sites,session_timespan,start_hour,day_of_week,time_of_day,weekend
session_id,,,,,,,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0,1,0,10,3,1,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518,7,0,11,5,1,1
3,14769,39,14768,14769,37,39,14768,14768,14768,14768,4,0,16,0,2,0
4,782,782,782,782,782,782,782,782,782,782,1,7,10,4,1,0
5,22,177,175,178,177,178,175,177,177,178,4,7,10,4,1,0


In [17]:
test_for_vw.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,#unique_sites,session_timespan,start_hour,day_of_week,time_of_day,weekend
session_id,,,,,,,,,,,,,,,,
1,29,35,22,321,23,2211,6730,21,44582,15336,10,0,11,5,1,1
2,782,782,782,782,782,782,782,782,782,782,1,2,11,3,1,0
3,55,55,55,55,55,55,55,55,1445,1445,2,2,15,4,2,0
4,1023,1022,50,222,202,3374,50,48,48,3374,7,0,10,1,1,0
5,301,301,301,66,67,69,70,68,71,167,8,0,15,4,2,0


**Реализуем функцию, `dataframe_to_vw`, переводящую обучающую или тестовую выборку в формат Vowpal Wabbit.**

Вход:
 - df – датафрейм `DataFrame` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам датафрейма `df` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [21]:
def dataframe_to_vw(df, y=None, out_file=os.path.join(PATH_TO_DATA,'tmp.vw')):
    df = df.astype(str)
    sites = ['site%d' % i for i in range(1, 11)]
    if y is None:
        y = ['1'] * df.shape[0]
    with open(out_file, 'w') as vw_data:
        for (_, row), target in zip(df.iterrows(), y):
            sample = str(target) + \
                    ' |a_sites ' + ' '.join([row[site] for site in sites if row[site] != '0']) + \
                    ''.join([' |' + col + ' ' + row[col] for col in row.index if col not in sites]) + \
                    '\n'
            vw_data.write(sample)
    pass

Проверяем:

In [22]:
dataframe_to_vw(train_for_vw_testing, y, out_file=os.path.join(PATH_TO_DATA,'tmp.vw'))

In [23]:
!head -3 $PATH_TO_DATA/tmp.vw

-1 |a_sites 718 |#unique_sites 1 |session_timespan 0 |start_hour 10 |day_of_week 3 |time_of_day 1 |weekend 0
-1 |a_sites 890 941 3847 941 942 3846 3847 3846 1516 1518 |#unique_sites 7 |session_timespan 0 |start_hour 11 |day_of_week 5 |time_of_day 1 |weekend 1
-1 |a_sites 14769 39 14768 14769 37 39 14768 14768 14768 14768 |#unique_sites 4 |session_timespan 0 |start_hour 16 |day_of_week 0 |time_of_day 2 |weekend 0


Всё корректно.    

### 3.2. Валидация по отложенной выборке

Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.

In [19]:
train_share = int(.7 * train_for_vw.shape[0])
train_for_vw_part = train_for_vw.iloc[:train_share, :]
valid_for_vw = train_for_vw.iloc[train_share:, :]

In [20]:
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

Сохраним вектора целевых функций, это может потребоваться в дальнейшем при **валидации**

In [22]:
np.save(os.path.join(PATH_TO_DATA,'vw_y_for_vw.npy'), y_for_vw)
np.save(os.path.join(PATH_TO_DATA,'vw_y_train_part_for_vw.npy'), y_train_part_for_vw)
np.save(os.path.join(PATH_TO_DATA,'vw_y_valid_for_vw.npy'), y_valid_for_vw)

#### Применим написанную функцию к части обучащей выборки (train_df_part, y_train_part_for_vw), к отложенной выборке (valid_df, y_valid_for_vw), ко всей обучающей выборке и ко всей тестовой выборке - всего 4 вызова.

In [43]:
%%time
dataframe_to_vw(train_for_vw_part, y_train_part_for_vw, out_file=os.path.join(PATH_TO_DATA,'vw_train_part.vw'))
dataframe_to_vw(valid_for_vw, y_valid_for_vw, out_file=os.path.join(PATH_TO_DATA,'vw_valid_part.vw'))
dataframe_to_vw(train_for_vw, y_for_vw, out_file=os.path.join(PATH_TO_DATA,'vw_train.vw'))
dataframe_to_vw(test_for_vw, out_file=os.path.join(PATH_TO_DATA,'vw_test.vw'))

Wall time: 5min 4s


Проверим:

In [44]:
!head -3 $PATH_TO_DATA/vw_valid_part.vw

-1 |a_sites 3000 32 33 35 3000 5302 23 21 5302 21 |#unique_sites 7 |session_timespan 3 |start_hour 13 |day_of_week 2 |time_of_day 2 |weekend 0
-1 |a_sites 21 39 676 812 23 679 570 814 21 39 |#unique_sites 8 |session_timespan 1 |start_hour 11 |day_of_week 1 |time_of_day 1 |weekend 0
-1 |a_sites 786 786 782 782 782 786 786 782 782 782 |#unique_sites 2 |session_timespan 1 |start_hour 8 |day_of_week 3 |time_of_day 1 |weekend 0


**Обучим модель Vowpal Wabbit на выборке `vw_train_part.vw`. Укажем, что решается задача бинарной классификации (`--binary`), сделаем 3 прохода по выборке (`--passes`). Поскольку для вычисления `roc_auc` нам потребуются вероятности, то функцию потерь (параметр `--loss_function`) возьмем `logistic`. Зададим некоторый кэш-файл (флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удалим с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, попробуем больше, чем 18 по умолчанию. Укажите `random_seed`=17. Остальные параметры пока оставим по умолчанию, далее, возможно, имеет смысл попробовать другие функции потерь.**

In [45]:
%%time
!vw --binary -d $PATH_TO_DATA/vw_train_part.vw --passes 3 -c -k --loss_function logistic \
-f $PATH_TO_DATA/vw_model_bin1.vw -b 26 --random_seed 17

Wall time: 3.29 s


final_regressor = data/vw_model_bin1.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = data/vw_train_part.vw.cache
Reading datafile = data/vw_train_part.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0  -1.0000  -1.0000        8
0.000000 0.000000            2            2.0  -1.0000  -1.0000       17
0.000000 0.000000            4            4.0  -1.0000  -1.0000       17
0.000000 0.000000            8            8.0  -1.0000  -1.0000       17
0.000000 0.000000           16           16.0  -1.0000  -1.0000       17
0.000000 0.000000           32           32.0  -1.0000  -1.0000       17
0.000000 0.000000           64           64.0  -1.0000  -1.0000       17
0.000000 0.000000          128          128.0  -1.0000  -1.0000       17
0.003906 0.007813          256

Сделаем предсказание и запишем прогнозы в **`vw_bin_train_part1.vw`**

In [46]:
%%time
!vw -t -i $PATH_TO_DATA/vw_model_bin1.vw \
-d $PATH_TO_DATA/vw_valid_part.vw \
-p $PATH_TO_DATA/vw_valid_bin_pred1.csv --random_seed 17

Wall time: 764 ms


only testing
predictions = data/vw_valid_bin_pred1.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = data/vw_valid_part.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
9.543870 9.543870            1            1.0  -1.0000  -4.0893       17
125.824303 242.104736            2            2.0  -1.0000 -16.5597       17
168.263796 210.703289            4            4.0  -1.0000 -12.0331       17
159.823450 151.383103            8            8.0  -1.0000 -11.2454       17
130.448373 101.073297           16           16.0  -1.0000 -11.2036       17
140.540938 150.633502           32           32.0  -1.0000  -7.7768       17
140.709853 140.878769           64           64.0  -1.0000 -10.0703       12
141.077516 141.445178          128          128.0  -1.0000 -11.6037       17
160.197752 179.317989          256          25

Считаем предсказания и посчитаем **roc_auc**

In [47]:
vw_valid_bin_pred1 = np.loadtxt(os.path.join(PATH_TO_DATA,'vw_valid_bin_pred1.csv'))

In [49]:
roc_auc_score(y_valid_for_vw, vw_valid_bin_pred1)

0.98484476413398214

Сделаем посылку на **Kaggle** по результатам работы **Vowpal Wabbit**.

In [90]:
# function for Kaggle submission
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [336]:
%%time
# ПОСЫЛКА НА Kaggle

# обучем на всей обучающей выборке
!vw --binary -d $PATH_TO_DATA/vw_train.vw --passes 3 -c -k --loss_function logistic \
-f $PATH_TO_DATA/vw_model_bin_train_all1.vw -b 26 --random_seed 17 --quiet

# предсказываем на тесте
!vw -t -i $PATH_TO_DATA/vw_model_bin_train_all1.vw \
-d $PATH_TO_DATA/vw_test.vw \
-p $PATH_TO_DATA/vw_test_bin_pred1.csv --random_seed 17 --quiet

# считываем предсказания
vw_test_bin_pred1 = np.loadtxt(os.path.join(PATH_TO_DATA,'vw_test_bin_pred1.csv'))

# формируем файл посылки на Kaggle
write_to_submission_file(vw_test_bin_pred1, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_vw_logistic1.csv'))

Wall time: 7.56 s


**Kaggle: 0.94275**.    
В общем то для начальной посылки неплохо, если учесть, что мы еще не подбирали параметры. А если учесть** небольшое** время работы алгоритма - то плюсы очевидны!

### 3.3. Подбор гиперпараметров

### 1 этап: Делаем подбор некоторых параметров вручную

Для удобства скомпануем код **более компактно**, сформировав некий **блок валидации** для **тестирования** некоторых параметров, вывод текущей информации тоже уберем.

In [3]:
# при необходимости считаем вектор ответов для валидации
y_valid_for_vw = np.load(os.path.join(PATH_TO_DATA,'vw_y_valid_for_vw.npy'))

In [10]:
%%time
# БЛОК ВАЛИДАЦИИ

# обучение на train_part
!vw --binary -d $PATH_TO_DATA/vw_train_part.vw --passes 9 -c -k --loss_function logistic \
-f $PATH_TO_DATA/vw_model_bin_tmp.vw -b 26 --random_seed 17 --quiet -q as

# предсказание на valid_part
!vw -t -i $PATH_TO_DATA/vw_model_bin_tmp.vw \
-d $PATH_TO_DATA/vw_valid_part.vw \
-p $PATH_TO_DATA/vw_valid_bin_pred_tmp.csv --random_seed 17 --quiet

# считаем предсказания и вычислим roc_auc
vw_valid_bin_pred_tmp = np.loadtxt(os.path.join(PATH_TO_DATA,'vw_valid_bin_pred_tmp.csv'))
print('roc_auc:', roc_auc_score(y_valid_for_vw, vw_valid_bin_pred_tmp))

roc_auc: 0.987711989314
Wall time: 9.56 s


Пробуем вручную параметры:
- **--passes**: улучшение есть (при увеличении), но повышать больше **9** смысла уже не имеет **[+]**
- **--loss_function**: **hinge** дает результат не очень хороший, оставляем **logistic**
- **-b** 28 сильно **перегружает** ресурсы и особенно память, а результат не увеличивает. Оставляем **26**
- **--ngram**: улучшения нет
- **-q**: as - немного улучшило

На этом ручной подбор в некотором роде исчерпан, далее будем подбирать с помощью библиотеки **hyperopt**.

### 2 этап: подбор гиперпараметров с помощью модуля `hyperopt`

Импортируем нужные элементы библиотеки **hyperopt**

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [94]:
%%time
def hyperopt_objective(params):
    # обучение на train_part
    !vw --binary -d $PATH_TO_DATA/vw_train_part.vw --passes 9 -c -k --loss_function logistic \
    -f $PATH_TO_DATA/vw_model_bin_tmp.vw -b 26 --random_seed 17 --quiet -q as \
    --learning_rate {params['learning_rate']} \
    --power_t {params['power_t']} \
    --initial_t {params['initial_t']} \
    --l1 {params['l1']} \
    --l2 {params['l2']}
    
    # предсказание на valid_part
    !vw -t -i $PATH_TO_DATA/vw_model_bin_tmp.vw \
    -d $PATH_TO_DATA/vw_valid_part.vw \
    -p $PATH_TO_DATA/vw_valid_bin_pred_tmp.csv --random_seed 17 --quiet
      
    # scoring roc_auc
    vw_valid_bin_pred_tmp = np.loadtxt(os.path.join(PATH_TO_DATA,'vw_valid_bin_pred_tmp.csv'))
    roc_auc_scoring = roc_auc_score(y_valid_for_vw, vw_valid_bin_pred_tmp)
    
    return 1 - roc_auc_scoring

# задаем диапазоны подбора параметров
params_space = {
    'learning_rate': hp.uniform('learning_rate', 0., 2.0), # default = 0.5
    'power_t': hp.uniform('power_t', 0., 2.0), # default = 0.5
    'initial_t': hp.loguniform('initial_t', -100, 1), # default = 0
    'l1': hp.loguniform('l1', -100, -10), # default = 0
    'l2': hp.loguniform('l2', -100, -10) # default = 0
                }

# запускаем подбор
trials = Trials()

best_params = fmin(
    hyperopt_objective,
    space=params_space,
    algo=tpe.suggest,
    max_evals=100,
    trials=trials
)

Wall time: 23min 9s


In [95]:
best_params

{'initial_t': 1.1505114803794776e-29,
 'l1': 1.8335143745961346e-31,
 'l2': 8.016379503987476e-29,
 'learning_rate': 0.42659194007671675,
 'power_t': 0.3974167436688515}

Проверим подобранные параметры на отложенной выборке:

In [96]:
%%time
# БЛОК ВАЛИДАЦИИ

# обучение на train_part
!vw --binary -d $PATH_TO_DATA/vw_train_part.vw --passes 9 -c -k --loss_function logistic \
-f $PATH_TO_DATA/vw_model_bin_tmp.vw -b 26 --random_seed 17 --quiet -q as \
--learning_rate {best_params['learning_rate']} \
--power_t {best_params['power_t']} \
--initial_t {best_params['initial_t']} \
--l1 {best_params['l1']} \
--l2 {best_params['l2']}

# предсказание на valid_part
!vw -t -i $PATH_TO_DATA/vw_model_bin_tmp.vw \
-d $PATH_TO_DATA/vw_valid_part.vw \
-p $PATH_TO_DATA/vw_valid_bin_pred_tmp.csv --quiet

# считаем предсказания и вычислим roc_auc
vw_valid_bin_pred_tmp = np.loadtxt(os.path.join(PATH_TO_DATA,'vw_valid_bin_pred_tmp.csv'))
print('roc_auc:', roc_auc_score(y_valid_for_vw, vw_valid_bin_pred_tmp))

roc_auc: 0.988443332364
Wall time: 19.1 s


Как и ожидалось, подбор параметров дал свой результат и метрика **roc_auc** на **валидации** заметно выросла.    

Сделаем обучение/предсказание на **всей** обучающей выборке и сделаем посылку на **Kaggle**. Практика показывает, что улучшение метрики на валидации не всегда приводит к улучшению метрики на **Kaggle**.

In [97]:
%%time
# ПОСЫЛКА НА Kaggle

# обучем на всей обучающей выборке
!vw --binary -d $PATH_TO_DATA/vw_train.vw --passes 9 -c -k --loss_function logistic \
-f $PATH_TO_DATA/vw_model_bin_train_param.vw -b 26 --random_seed 17 --quiet -q as \
--learning_rate {best_params['learning_rate']} \
--power_t {best_params['power_t']} \
--initial_t {best_params['initial_t']} \
--l1 {best_params['l1']} \
--l2 {best_params['l2']}

# предсказываем на тесте
!vw -t -i $PATH_TO_DATA/vw_model_bin_train_param.vw \
-d $PATH_TO_DATA/vw_test.vw \
-p $PATH_TO_DATA/vw_model_bin_train_param_pred.csv --quiet

# считываем предсказания
vw_model_bin_train_param_pred = np.loadtxt(os.path.join(PATH_TO_DATA,'vw_model_bin_train_param_pred.csv'))

# формируем файл посылки на Kaggle
write_to_submission_file(vw_model_bin_train_param_pred, os.path.join(PATH_TO_DATA, 
                                                                     '[YDF & MIPT]_Coursera_vw_logistic_hyperopt2.csv'))

Wall time: 22.2 s


**Kaggle:** 0.93547

Как мы увидели, результат на **Kaggle** даже немного уменьшился.    

**РЕЗЮМЕ:**   
- **Vopal Wabbit** - достаточно эффективная и быстрая библиотека
- **hyperopt** - удобный модуль для подбора гиперпараметров